In [1]:
import numpy as np
from tqdm import tqdm
from solver.qp import qp
import pandas as pd
import torch
import gzip
import pickle
import warnings
from generate_instances import generate_softmarginsvm, generate_markowitz_portfolio_optimization, generate_convex_function_to_data, generate_regressor_selection

### Soft margin Support Vector Machine

In [2]:
def surrogate_gen():
    NUM = 500 # Number of points
    DIM = 2 # Dimension of the points
    C = 10 # regularization parameter (hardness of the margin)
    
    
    # 2-D mean of ones
    M1 = np.ones((DIM,))
    # 2-D mean of threes
    M2 = 3 * np.ones((DIM,))
    # 2-D covariance of 0.3
    C1 = np.diag(0.3 * np.ones((DIM,)))
    # 2-D covariance of 0.2
    C2 = np.diag(0.2 * np.ones((DIM,)))

    # generate 50 points from gaussian 1
    x1 = np.random.multivariate_normal(M1, C1, NUM) #generate gaussian
    # labels
    y1 = np.ones((x1.shape[0],))
    # generate 50 points from gaussian 2
    x2 = np.random.multivariate_normal(M2, C2, NUM)
    y2 = -np.ones((x2.shape[0],))
    # join
    X = np.concatenate((x1, x2), axis = 0)
    y = np.concatenate((y1, y2), axis = 0)

    Q, q, G, h, A, b, S = generate_softmarginsvm(y, X, C)    
    return Q, q, G, h, A, b, S

### Markowitz portfolio optimization

In [4]:
def surrogate_gen():
    np.random.seed(1)
    n_obs = 252
    n_assets = 4
    artificial_returns = np.random.randn(n_obs, n_assets) + 0.05 # minimum expected return
    returns = pd.DataFrame(artificial_returns)
    pbar = np.array(returns.mean()).reshape(1, -1)
    r_min = pbar.mean() # minimum return threshold
    Q, q, G, h, A, b, S = generate_markowitz_portfolio_optimization(artificial_returns, r_min, pbar)
    return Q, q, G, h, A, b, S

### Fiting convex function to given data

In [23]:
def surrogate_gen():
    
    # Generate 'u' values (from -2 to 2) for the independent variable
    NUM = 4
    u_values = np.linspace(-2.0, 2.0, num=NUM).reshape(-1, 1)
    Q, q, G, h, A, b, S = generate_convex_function_to_data(u_values)
    return Q, q, G, h, A, b, S

### Regressor selection problem

In [26]:
def surrogate_gen():
    
    # Set dimensions
    m = 10  # number of rows for A
    n = 20  # number of columns for A
    nopts = 100 # range of alpha values
    alpha_index = np.random.randint(0, nopts) # Select an alpha value from the range nopts
    Q, q, G, h, A, b, S = generate_regressor_selection(alpha_index, m, n)
    
    return Q, q, G, h, A, b, S

### create

In [5]:
import os

# directory = f'../../../../work/log1/darius.weber/Quadratic_Programming_Datasets/raw'
directory = f'Quadratic_Programming_Datasets/raw'
os.makedirs(directory, exist_ok=True)
warnings.filterwarnings("error")

ips = []
pkg_idx = 0 #identifier index
success_cnt = 0
fail_cnt = 0
max_ipm_steps = 0


max_iter = 100
num = 10

for i in tqdm(range(max_iter)):
    Q, q, G, h, A, b, S = surrogate_gen()
    try:
        res = qp(Q, q, G, h, A, b, callback=lambda res: res)
        max_ipm_steps = max(max_ipm_steps, res['iterations'])
        print("status",res['status'])
    except Exception as e:
        fail_cnt += 1
        warnings.warn(f'Optimization failed with error: {str(e)}')
        continue
    else:
        if res['status'] == 'optimal': #np.isnan(res.fun)?
            ips.append((torch.from_numpy(np.array(Q)).to(torch.float), torch.from_numpy(np.array(q)).to(torch.float), torch.from_numpy(np.array(G)).to(torch.float), torch.from_numpy(np.array(h)).to(torch.float), torch.from_numpy(np.array(A)).to(torch.float), torch.from_numpy(np.array(b)).to(torch.float), torch.from_numpy(np.array(S)).to(torch.float), res))
            success_cnt += 1
            
    if len(ips) >= 1000 or success_cnt == num:
        print("success")
        ips = [ip + (max_ipm_steps,) for ip in ips]
        with gzip.open(f'{directory}/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            print(f'Package {pkg_idx} saved')
            pkg_idx += 1

        ips = []
    if success_cnt >= num:
        break
warnings.resetwarnings()

  9%|▉         | 9/100 [00:00<00:03, 23.07it/s]

     pcost       dcost       gap    pres   dres
 0:  1.1389e-01 -8.9748e-01  6e+00  2e+00  3e+00
 1:  1.1513e-01 -5.3957e-01  7e-01  2e-02  2e-02
 2:  1.1519e-01  1.0276e-01  1e-02  3e-04  3e-04
 3:  1.1415e-01  1.1368e-01  5e-04  5e-06  6e-06
 4:  1.1396e-01  1.1395e-01  2e-05  5e-08  6e-08
 5:  1.1395e-01  1.1395e-01  2e-07  5e-10  6e-10
 6:  1.1395e-01  1.1395e-01  2e-09  5e-12  6e-12
Optimal solution found.
status optimal
     pcost       dcost       gap    pres   dres
 0:  1.1389e-01 -8.9748e-01  6e+00  2e+00  3e+00
 1:  1.1513e-01 -5.3957e-01  7e-01  2e-02  2e-02
 2:  1.1519e-01  1.0276e-01  1e-02  3e-04  3e-04
 3:  1.1415e-01  1.1368e-01  5e-04  5e-06  6e-06
 4:  1.1396e-01  1.1395e-01  2e-05  5e-08  6e-08
 5:  1.1395e-01  1.1395e-01  2e-07  5e-10  6e-10
 6:  1.1395e-01  1.1395e-01  2e-09  5e-12  6e-12
Optimal solution found.
status optimal
     pcost       dcost       gap    pres   dres
 0:  1.1389e-01 -8.9748e-01  6e+00  2e+00  3e+00
 1:  1.1513e-01 -5.3957e-01  7e-01  2e-02  

In [ ]:
# Open and load the .pkl.gz file
with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{pkg_idx-1}.pkl.gz', 'rb') as f:
    data = pickle.load(f)

# Now `data` contains the contents of the pickle file
print(data)